In [50]:
# imports
from matplotlib import pyplot as plt
import pandas as pd
from hmmlearn import hmm
import numpy as np
import sys
import time
sys.path.insert(1, '/mnt/c/Users/dalli/source/acme_senior/_projectV3/CarKalmanFilter/')


import cleaner
import filter

wsl = True

The following tables and images are from https://www.kaggle.com/code/jefmenegazzo/road-surface-type-classification

| Hardware       | Sensor         | Data                                 | Sampling Rate |
|----------------|----------------|--------------------------------------|---------------|
| HP Webcam HD-4110 | Camera      | 720p Video                           | 30 Hz         |
| Xiaomi Mi 8       | GPS         | Speed in m/s, latitude, longitude, etc. | 1 Hz       |
| MPU-9250          | Accelerometer | 3-axis acceleration in m/s²         | 100 Hz        |
| MPU-9250          | Gyroscope    | 3-axis rotation rate in deg/s        | 100 Hz        |
| MPU-9250          | Magnetometer | 3-axis ambient geomagnetic field in µT | 100 Hz       |
| MPU-9250          | Temperature  | Sensor temperature in ◦C              | 100 Hz       |

| DataSet | Vehicle              | Driver   | Scenario  | Distance |
|---------|----------------------|----------|-----------|----------|
| PVS 1   | Volkswagen Saveiro   | Driver 1 | Scenario 1| 13.81 km |
| PVS 2   | Volkswagen Saveiro   | Driver 1 | Scenario 2| 11.62 km |
| PVS 3   | Volkswagen Saveiro   | Driver 1 | Scenario 3| 10.72 km |
| PVS 4   | Fiat Bravo           | Driver 2 | Scenario 1| 13.81 km |
| PVS 5   | Fiat Bravo           | Driver 2 | Scenario 2| 11.63 km |
| PVS 6   | Fiat Bravo           | Driver 2 | Scenario 3| 10.73 km |
| PVS 7   | Fiat Palio           | Driver 3 | Scenario 1| 13.78 km |
| PVS 8   | Fiat Palio           | Driver 3 | Scenario 2| 11.63 km |
| PVS 9   | Fiat Palio           | Driver 3 | Scenario 3| 10.74 km |

| File                   | Description                                                                                       |
|------------------------|---------------------------------------------------------------------------------------------------|
| dataset_gps.csv        | GPS data, including latitude, longitude, altitude, speed, accuracy, etc.                         |
| dataset_gps_mpu_left.csv  | Inertial sensor data on the left side of the vehicle, combined with GPS data.                     |
| dataset_gps_mpu_right.csv | Inertial sensor data on the right side of the vehicle, combined with GPS data.                    |
| dataset_labels.csv       | Data classes for each sample data in the dataset (for both sides).                                  |
| dataset_mpu_left.csv     | Inertial sensor data on the left side of the vehicle.                                              |
| dataset_mpu_right.csv    | Inertial sensor data on the right side of the vehicle.                                             |

![Map of paths](./maps.png "All paths driven by the three drivers.")


In [51]:
if wsl:
    parent = '/mnt/c/Users/dalli/source/acme_senior/_projectV3/CarKalmanFilter/.data'
else:
    parent = ".data"
data_dict = cleaner.clean_dict(cleaner.load_data(parent, exclude_test=["PVS 7", "PVS 8"], exclude_val=["PVS 9"]))

In [52]:
cleaner.print_structure(data_dict)


  train: 
 	 t_gps: 
 		 PVS 1: (1458, 11)
 		 PVS 2: (1551, 11)
 		 PVS 3: (1316, 11)
 		 PVS 4: (1432, 11)
 		 PVS 5: (1263, 11)
 		 PVS 6: (915, 11)
 	 gps_mpu_left: 
 		 PVS 1: (144036, 26)
 		 PVS 2: (124684, 26)
 		 PVS 3: (105816, 26)
 		 PVS 4: (132492, 26)
 		 PVS 5: (133877, 26)
 		 PVS 6: (96279, 26)
 	 gps_mpu_right: 
 		 PVS 1: (144036, 26)
 		 PVS 2: (124684, 26)
 		 PVS 3: (105816, 26)
 		 PVS 4: (132492, 26)
 		 PVS 5: (133877, 26)
 		 PVS 6: (96279, 26)
 	 labels: 
 		 PVS 1: (144036, 14)
 		 PVS 2: (124684, 14)
 		 PVS 3: (105816, 14)
 		 PVS 4: (132492, 14)
 		 PVS 5: (133877, 14)
 		 PVS 6: (96279, 14)
 	 folders: 6
  val: 
 	 t_gps: 
 		 PVS 9: (999, 11)
 	 gps_mpu_left: 
 		 PVS 9: (91555, 26)
 	 gps_mpu_right: 
 		 PVS 9: (91555, 26)
 	 labels: 
 		 PVS 9: (91555, 14)
 	 folders: 1
  test: 
 	 t_gps: 
 		 PVS 7: (1281, 11)
 		 PVS 8: (1134, 11)
 	 gps_mpu_left: 
 		 PVS 7: (128548, 26)
 		 PVS 8: (123618, 26)
 	 gps_mpu_right: 
 		 PVS 7: (128548, 26)
 		 PVS 8:

In [ ]:
filter.add_smoothed_cols(data_dict, window=200)

In [64]:
data_dict["train"]["labels"]["PVS 1"][["good_road_left", "regular_road_left", "bad_road_left"]].describe()

,good_road_left,regular_road_left,bad_road_left
count,144036.000000,144036.000000,144036.000000
mean,0.392798,0.453387,0.153816
std,0.488374,0.497824,0.360773
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000


In [55]:
def build_z_dash(df, type="train", road="good_road_left"):
    """
    Builds a 1 column dataset from the given dataframe

    :param df: dataframe
    :param type: string

    :return: dataframe with only the acc_z_dash column
    :return: list of lengths
    """
    result = None
    lengths = []
    for folder in df[type]["gps_mpu_right"].keys():
        pvsi = df[type]["gps_mpu_right"][folder]
        labels = df[type]["labels"][folder][road]
        indices = labels[labels == 1].index

        new_data = pvsi["acc_z_dash"].loc[indices] + pvsi["acc_z_dash_smooth"].loc[indices]
        lengths.append(len(new_data))

        if result is None:
            result = new_data
        else:
            # append on the same column
            result = pd.concat([result, new_data], axis=0)

    return result, lengths


train_dict = {
    "good": build_z_dash(data_dict, type="train", road="good_road_left"),
    "regular": build_z_dash(data_dict, type="train", road="regular_road_left"),
    "bad": build_z_dash(data_dict, type="train", road="bad_road_left")
}

test_dict = {
    "good": build_z_dash(data_dict, type="test", road="good_road_left"),
    "regular": build_z_dash(data_dict, type="test", road="regular_road_left"),
    "bad": build_z_dash(data_dict, type="test", road="bad_road_left")
}

# HMM

I got it to run (it turns out that you can only include one series as your input data) but it doesn't work. I think the HMM decided that zeros are local minimums or values below a threshold and pretty much everything else is ones.

In [56]:
models = {}
num_models = 3
for key in train_dict.keys():
    data, lengths = train_dict[key]
    # make data an array
    data = data.values.reshape(-1, 1)
    # hmmlearn expects the data to be in a single array:
    # To separate the sequences, it requires the length of each:
    print("\nTraining for", key, end=": ")

    best_log = -np.inf
    for i in range(num_models):
        # Initialize and train the model
        model = hmm.GMMHMM(n_components=6, covariance_type="diag")
        model.fit(data, lengths=lengths)
        # Check the log-likelihood
        log_likelihood = model.monitor_.history[-1]
        if log_likelihood > best_log:
            best_log = log_likelihood
            best_model = model
        print(i, end=", ")
    models[key] = best_model


Training for good: 0, 1, 2, 
Training for regular: 0, 1, 2, 
Training for bad: 0, 1, 2, 

In [59]:
# predicted_labels = model.predict(X)

def predict(mfcc_coeffs):
    """
    Predict the word from the given mfcc coefficients
    
    Parameters
    ----------
    mfcc_coeffs : ndarray of shape (M,)
        The mfcc coefficients for the word to be predicted
        
    Returns
    -------
    word : str
        The predicted word
    """
    # find the log probability density of the given mfcc coefficients
    log_prob = {}
    for key in models.keys():
        log_prob[key] = models[key].score(mfcc_coeffs)
    
    # return the word with the highest probability
    return max(log_prob, key=log_prob.get)



time_window = 10 # seconds
time_window *= 1000
for key in test_dict.keys():
    correct = 0
    incorrect = 0
    test_data, _ = test_dict[key]

    # iterate over test_data in groups of 10000
    test_data = test_data.values.reshape(-1, 1)
    for i in range(0, len(test_data), time_window):
        item = test_data[i:i+time_window]
        prediction = predict(item)
        if prediction == key:
            correct += 1
        elif prediction == "good" and key == "regular":
            correct += 1
        elif prediction == "regular" and key == "good":
            correct += 1
        else:
            incorrect += 1
    
    print("Accuracy for", key, ":", correct/(correct+incorrect))

Accuracy for good : 0.7272727272727273
Accuracy for regular : 0.0
Accuracy for bad : 1.0


In [58]:
# how many times does predicted_labels switch form 0 to 1
switches = 0
for i in range(1, len(predicted_labels)):
    if predicted_labels[i] != predicted_labels[i-1]:
        switches += 1

print(f"Switches: {switches}")
print(f"Lenght of labels: {len(predicted_labels)}") 
print(f"Ratio: {switches/len(predicted_labels)}")

# plot the predicted labels
start = 20000
end = 21000
diff = 1000
plt.figure(figsize=(20, 20))
p = 1
while p < 20:
    plt.subplot(5, 4, max(p,1))
    p += 1
    start += diff
    end += diff   
    a = predicted_labels[start:end]
    # count how many zeros
    zeros = np.count_nonzero(a == 0)
    # print("Zeros: ", zeros)
    # print("P:", p)
    if zeros == 0:
        p -= 1
        continue
    plt.scatter(np.arange(diff), predicted_labels[start:end])
    plt.plot(np.arange(diff), hmm_true['paved_road'][start:end], color='red')
    plt.plot(np.arange(diff), X[start:end, 0]-9.5, color='green')
    plt.title(f"Second {end/1000}, Zeros: {zeros}")
    # remove axis ticks
    # plt.xticks([])
    # plt.yticks([])

# reduce space between subplots
plt.tight_layout()
plt.show()

NameError: name 'predicted_labels' is not defined